# Step 3-1. Bedrock Titan 임베딩으로 벡터 검색
Bedrock Titan 텍스트 임베딩 모델을 사용하여 k-NN 유사도 검색을 수행합니다.

In [ ]:
!pip install -q boto3==1.38.46 opensearch-py==2.8.0

## 1. 설정 (Configuration)

In [ ]:
import os, json

# Step 0에서 저장한 설정 불러오기
try:
    with open("../config.json") as f:
        _config = json.load(f)
    print("✅ config.json 로드 완료")
except FileNotFoundError:
    raise FileNotFoundError("❌ config.json을 찾을 수 없습니다. Step 0 노트북을 먼저 실행해주세요.")

HOST = _config.get("OPENSEARCH_HOST")
if not HOST:
    raise ValueError("❌ config.json에 OPENSEARCH_HOST 값이 없습니다. Step 0 노트북을 먼저 실행해주세요.")
DEFAULT_REGION = _config.get("DEFAULT_REGION", "ap-northeast-2")
BEDROCK_REGION = _config.get("BEDROCK_REGION", "us-east-1")
PROFILE = _config.get("PROFILE", "skku-opensearch-session")

INDEX_NAME = 'bedrock-test'
EMBEDDING_MODEL_ID = 'amazon.titan-embed-text-v2:0'

## 2. OpenSearch 클라이언트 생성

In [2]:
import boto3
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection

service = 'aoss'
credentials = boto3.Session(profile_name=PROFILE).get_credentials()
auth = AWSV4SignerAuth(credentials, DEFAULT_REGION, service)

client = OpenSearch(
    hosts=[{'host': HOST, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

print("OpenSearch 클라이언트 생성 완료")

OpenSearch 클라이언트 생성 완료


## 3. Bedrock 클라이언트 생성 및 임베딩 함수 정의

In [3]:
import json

print(f"Creating a boto3 session with profile '{PROFILE}'...")
session = boto3.Session(profile_name=PROFILE)

print(f"Creating a Bedrock client in region: {BEDROCK_REGION}")
bedrock_client = session.client(
    service_name='bedrock-runtime',
    region_name=BEDROCK_REGION,
)
print("Bedrock client created successfully.")

def get_embedding_from_bedrock(text, model_id):
    """Bedrock API를 호출하여 주어진 텍스트의 벡터 임베딩을 반환합니다."""
    body = json.dumps({"inputText": text})
    response = bedrock_client.invoke_model(
        body=body,
        modelId=model_id,
        accept="application/json",
        contentType="application/json",
    )
    response_body = json.loads(response.get("body").read())
    return response_body.get("embedding")

Creating a boto3 session with profile 'skku-opensearch-session'...
Creating a Bedrock client in region: us-east-1
Bedrock client created successfully.


In [ ]:
# 인덱스에 데이터가 업로드되었는지 확인
if not client.indices.exists(index=INDEX_NAME):
    print(f"❌ '{INDEX_NAME}' 인덱스가 존재하지 않습니다.")
    print("   → Step 3-0 (Titan 임베딩 데이터 업로드) 노트북을 먼저 실행해주세요.")
else:
    _doc_count = client.count(index=INDEX_NAME)['count']
    if _doc_count == 0:
        print(f"⏳ '{INDEX_NAME}' 인덱스는 있지만 검색 가능한 문서가 0개입니다.")
        print("   → 데이터 업로드 직후라면 인덱싱 중일 수 있습니다. 잠시 후 다시 실행해주세요.")
    else:
        print(f"✅ '{INDEX_NAME}' 인덱스에 {_doc_count}개의 문서가 준비되어 있습니다.")

## 4. 검색어 벡터 변환 및 k-NN 검색

In [ ]:
from opensearchpy.exceptions import RequestError

# TODO: 검색할 문장을 바꿔보세요!
QUERY_TEXT = "s3랑 cloudfront를 활용해서 정적 웹사이트를 배포하는 방법 알려줘"

print(f"Creating a vector for the query: '{QUERY_TEXT}'...")
query_vector = get_embedding_from_bedrock(QUERY_TEXT, EMBEDDING_MODEL_ID)

K_NEIGHBORS = 5
RESULT_SIZE = 5

search_query = {
    "size": RESULT_SIZE,
    "query": {
        "knn": {
            "content_vector": {
                "vector": query_vector,
                "k": K_NEIGHBORS
            }
        }
    }
}

print(f"Searching for top {RESULT_SIZE} similar documents in index '{INDEX_NAME}'...")

try:
    response = client.search(index=INDEX_NAME, body=search_query)
    hits = response['hits']['hits']
    print(f"\n--- '{QUERY_TEXT}'와(과) 유사한 문서 검색 결과({len(hits)}) ---")

    if not hits:
        print("검색된 문서가 없습니다.")
        print("팁: INDEX_NAME이 정확한지, 데이터가 정상적으로 업로드되었는지 확인해보세요.")
    else:
        for i, hit in enumerate(hits):
            score = hit['_score']
            title = hit['_source'].get('title', 'N/A')
            content = hit['_source'].get('content', 'N/A')
            print(f"\n[{i + 1}] 유사도: {score:.4f}")
            print(f"    제목: {title}")
            print(f"    내용: {content[:150]}...")

except RequestError as e:
    print("\n[!!!] 검색 중 에러가 발생했습니다.")
    print(f"상태 코드: {e.status_code}")
    print(f"에러 정보: {e.error}")
    print(f"에러 원인: {e.info.get('error', {}).get('root_cause', [{}])[0].get('reason', '알 수 없음')}")
except Exception as e:
    print(f"\n[!!!] 예상치 못한 에러가 발생했습니다: {e}")